In [2]:
import os
import numpy as np
np.random.seed(43)

In [341]:
setup_string = '''
# General setup
open morph-open-closed/open-closed.pdb
center
turn y 60

# Open bound structure get position of the ligand, missing in the morph
open 1ake
delete #1:.B
addh spec #1&protein hbond false
mmaker #0.1 #1

# Window appearance
windowsize 800 600
lighting reflectivity 0
set bg_color white
set silhouette
set silhouette_width 1.5

# Model appearance
color grey

# Add caption
2dlabel create title text "Binding and release of substrate leads to directional motion" xpos 0.1 ypos 0.92 color black

# Play movie sequence
2dlabel change title visibility show

# Hide everything
~ribbon; ~display
# Except ligand
display #1:AP5; color red #1:AP5; color byhet #1:AP5
'''

In [342]:
def pause_and_fade(pause_length, total_frames, file):
    file.write('# Fade out...\n')
    for pause in np.arange(pause_length):
        total_frames += 1
        transparency_increment = 100. / float(pause_length)
        file.write('transparency {} #1:AP5\n'.format((pause + 1)* transparency_increment)
                  )
        file.write('copy file morph-open-closed/open-closed-{:03.0f}.png\n'.format(total_frames)
        )
    file.write('~display #1:AP5\n')
    return total_frames

In [343]:
def pause_and_fade_in(pause_length, total_frames, file):
    file.write('# Fade in...\n')
    file.write('display #1:AP5\n')
    for pause in np.arange(pause_length):
        total_frames += 1
        transparency_increment = 100. / float(pause_length)
        file.write('transparency {} #1:AP5\n'.format(100 - ((pause + 1)* transparency_increment))
        )
        file.write('copy file morph-open-closed/open-closed-{:03.0f}.png\n'.format(total_frames)
        )

    return total_frames


def pause(pause_length, total_frames, file):
    for pause in np.arange(pause_length):
        total_frames += 1
        file.write('copy file morph-open-closed/open-closed-{:03.0f}.png\n'.format(total_frames)
        )
    return total_frames

In [362]:
def add_bump(positive):
    sigma = 2
    if positive:
        mu = 6.0
    else:
        mu = -6.0
    rotation = sigma * np.random.randn(1) + mu
    return rotation[0]

In [357]:
def rotate_forward(models, residues, positives, file, total_frames):
    forward_rotations = np.zeros(( len(residues), len(models) ))
    for model in models:
        file.write('~show #0; ~ribbon #0\n')
        for (residue_index, residue) in enumerate(residues): 
            # Add a bump to the previous value!
            forward_rotations[residue_index, model] = forward_rotations[residue_index, model - 1] + \
            add_bump(positives[residue_index])
            
            file.write('rotation 2 #0.{}:{}@CA,CB\n'.format(model + 1, residue))
            colors = ['blue', 'green', 'purple', 'orange', 'yellow']
            file.write('rotation 2 {} 1; wait 1\n'.format(forward_rotations[residue_index, model - 1] + \
            add_bump(positives[residue_index])))
            file.write('display #0.{}:{};'.format(model + 1, residue))
            file.write('color {} #0.{}:{};'.format(colors[residue_index], model + 1, residue))
            file.write('color byhet #0.{}:{}\n'.format(model + 1, residue))
            file.write('~rotation 2\n')

        file.write('ribbon #0.{}\n'.format(model + 1))
        total_frames += 1
        file.write('copy file morph-open-closed/open-closed-{:03.0f}.png \n\n'.format(total_frames)
        )

    return forward_rotations, total_frames

In [360]:
def rotate_backward(models, residues, positives, file, forward_rotations, total_frames):
    backward_rotations = np.zeros(( len(residues), len(models) ))
    for model in models[::-1]:
        file.write('~show #0; ~ribbon #0\n')
        for (residue_index, residue) in enumerate(residues):    
            # Find out how much this torsion needs to rotate to catch up...
            maximum_rotation = forward_rotations[residue_index, -1]
            rotation_necessary = maximum_rotation - forward_rotations[residue_index, model]

            backward_rotations[residue_index, model] = rotation_necessary + \
            (models[-1] - model) * add_bump(positives[residue_index])
            
            file.write('rotation 2 #0.{}:{}@CA,CB\n'.format(model + 1, residue))
            colors = ['blue', 'green', 'purple', 'orange', 'yellow']
            file.write('rotation 2 {} 1; wait 1\n'.format(rotation_necessary + \
            (models[-1] - model) * add_bump(positives[residue_index])))
            file.write('display #0.{}:{};'.format(model + 1, residue))
            file.write('color {} #0.{}:{};'.format(colors[residue_index], model + 1, residue))
            file.write('color byhet #0.{}:{}\n'.format(model + 1, residue))
            file.write('~rotation 2\n')

        file.write('ribbon #0.{}\n'.format(model + 1))
        total_frames += 1
        file.write('copy file morph-open-closed/open-closed-{:03.0f}.png \n\n'.format(total_frames)
            )
    
    return backward_rotations, total_frames

In [364]:
residues = [20, 50, 120, 141, 200]
positives = [True, False, False, True, True]
pause_length = 20
morphs = np.arange(30)
total_frames = 0


with open('morph_setup.cmd', 'w') as file:
    file.write(setup_string)

    forward_rotations, total_frames = rotate_forward(
        morphs,
        residues,
        positives, 
        file, 
        total_frames)
    
    total_frames = pause_and_fade(5, total_frames, file)
    
    backward_rotations, total_frames = rotate_backward(
        morphs, 
        residues, 
        positives, 
        file, 
        forward_rotations, 
        total_frames)